In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

class CSVCreditScorer:
    def __init__(self):
        """CSV 파일 전용 신용점수 계산기"""
        
        # 요인별 가중치 (기존과 동일)
        self.factor_weights = {
            'Factor_1': {
                'operating_expenses': 0.074535,
                'cost_of_goods_sold': 0.07436,
                'weekend_sales_amount': 0.074554,
                'total_sales_amount': 0.07453,
                'other_expenses': 0.073679,
                'total_salary': 0.0736,
                'rent_expense': 0.072611,
                'weekday_sales_amount': 0.072501,
                'avg_account_balance': 0.07072,
                'operating_profit': 0.069715,
                'weekday_transaction_count': 0.066501,
                'transaction_count': 0.063799,
                'weekend_transaction_count': 0.057164,
                'water_usage_ton': 0.025919,
                'electricity_bill_amount': 0.027752,
                'electricity_usage_kwh': 0.028061
            },
            'Factor_2': {
                'revisit_customer_sales_ratio': 0.083268,
                'sales_cv': 0.084229,
                'new_customer_ratio': 0.078371,
                'operating_profit_ratio': 0.093389,
                'operating_expense_ratio': 0.093389,
                'energy_eff_appliance_ratio': 0.04497,
                'cogs_ratio': 0.049744,
                'customer_review_avg_rating': 0.056357,
                'rent_payment_compliance_rate': 0.025489,
                'card_payment_ratio': 0.057726,
                'cash_payment_ratio': 0.057712,
                'salary_payment_regularity': 0.022855,
                'customer_review_positive_ratio': 0.034197,
                'min_balance_maintenance_ratio': 0.021572,
                'weighted_avg_cash_period': 0.056587,
                'cash_payment_ratio_detail': 0.058045,
                'card_payment_ratio_detail': 0.057432,
                'tax_payment_integrity': 0.012474,
                'cashflow_cv': 0.012193
            },
            'Factor_3': {
                'weekend_avg_transaction_value': 0.229004,
                'weekday_avg_transaction_value': 0.229004,
                'avg_transaction_value': 0.229004,
                'dinner_sales_ratio': 0.131089,
                'lunch_sales_ratio': 0.087794,
                'food_waste_kg_per_day': 0.094106
            },
            'Factor_4': {
                'electricity_usage_kwh': 0.191075,
                'food_waste_kg_per_day': 0.187428,
                'electricity_bill_amount': 0.185198,
                'water_usage_ton': 0.163306,
                'recycle_waste_kg_per_day': 0.160769,
                'gas_usage_m3': 0.112225
            },
            'Factor_5': {
                'cash_payment_ratio': 0.364838,
                'card_payment_ratio': 0.364834,
                'lunch_sales_ratio': 0.270328
            },
            'Factor_6': {
                'card_payment_ratio_detail': 0.341636,
                'cash_payment_ratio_detail': 0.339214,
                'weighted_avg_cash_period': 0.31915
            },
            'Factor_7': {
                'yellow_umbrella_member': 0.484789,
                'yellow_umbrella_amount': 0.362795,
                'yellow_umbrella_months': 0.152416
            },
            'Factor_8': {
                'operating_profit_ratio': 0.435107,
                'operating_expense_ratio': 0.435107,
                'salary_ratio': 0.129785
            }
        }
        
        # 요인별 중요도 (기존과 동일)
        self.factor_importance = {
            'Factor_1': 0.20, 'Factor_2': 0.25,
            'Factor_3': 0.25, 'Factor_4': 0.10,
            'Factor_5': 0.05, 'Factor_6': 0.10,
            'Factor_7': 0.03, 'Factor_8': 0.02
        }
        
        # 역방향 지표 (기존과 동일)
        self.reverse_indicators = {
            'sales_cv', 'cashflow_cv', 'cogs_ratio', 'rent_ratio', 'operating_expenses',
            'operating_expense_ratio', 'weighted_avg_cash_period', 'electricity_usage_kwh',
            'electricity_bill_amount', 'rent_expense', 'gas_usage_m3', 'water_usage_ton',
            'food_waste_kg_per_day', 'other_expenses', 'salary_ratio', 'other_payment_ratio',
            'excessive_withdrawal_frequency', 'origin_price_violation_count'
        }
        
        # 등급 매핑 (기존과 동일)
        self.grade_mapping = [
            (900, "AAA"), (800, "AA"), (700, "A"),
            (600, "BBB"), (500, "BB"), (400, "B"),
            (300, "CCC"), (200, "CC"), (0, "C")
        ]
        
        # 데이터 정규화를 위한 통계값 저장
        self.normalization_stats = {}
    
    def fit_normalization(self, df):
        """데이터 정규화를 위한 통계값 계산"""
        all_variables = set()
        for factor_vars in self.factor_weights.values():
            all_variables.update(factor_vars.keys())
        
        for variable in all_variables:
            if variable in df.columns:
                values = df[variable].replace([np.inf, -np.inf], np.nan).dropna()
                if len(values) > 0:
                    # 백분위수 기반 정규화를 위한 통계값
                    self.normalization_stats[variable] = {
                        'p10': values.quantile(0.1),
                        'p90': values.quantile(0.9),
                        'median': values.median(),
                        'mean': values.mean(),
                        'std': values.std()
                    }
                else:
                    # 기본값
                    self.normalization_stats[variable] = {
                        'p10': 0, 'p90': 1, 'median': 0.5, 'mean': 0.5, 'std': 0.25
                    }
    
    def normalize_value(self, variable, value):
        """개선된 변수별 점수 정규화"""
        if pd.isna(value) or np.isinf(value):
            return 50  # 중간값
        
        value = float(value)
        
        if variable not in self.normalization_stats:
            return 50
        
        stats = self.normalization_stats[variable]
        
        # 백분위수 기반 정규화 (10~90 백분위수를 20~80점에 매핑)
        if stats['p90'] != stats['p10']:  # 분모가 0이 아닌 경우
            normalized = (value - stats['p10']) / (stats['p90'] - stats['p10'])
            score = 20 + normalized * 60  # 20~80점 범위
        else:
            score = 50  # 모든 값이 같으면 중간값
        
        # 범위 조정
        score = max(10, min(90, score))
        
        # 역방향 지표 처리
        if variable in self.reverse_indicators:
            score = 100 - score
        
        return score
    
    def calculate_factor_score(self, factor_name, row_data):
        """요인 점수 계산 (기존과 동일)"""
        factor_vars = self.factor_weights.get(factor_name, {})
        if not factor_vars:
            return 50
        
        weighted_sum, total_weight = 0, 0
        
        for variable, weight in factor_vars.items():
            if variable in row_data:
                score = self.normalize_value(variable, row_data[variable])
                weighted_sum += score * weight
                total_weight += weight
        
        return weighted_sum / total_weight if total_weight > 0 else 50
    
    def calculate_single_score(self, row_data):
        """단일 행 신용점수 계산"""
        factor_scores = {
            f: self.calculate_factor_score(f, row_data)
            for f in self.factor_weights.keys()
        }
        
        # 최종 점수 계산 (가중평균)
        total_importance = sum(self.factor_importance.values())
        final_score = sum(
            (factor_scores[f] * self.factor_importance[f]) / total_importance
            for f in factor_scores
        )
        
        # 신용점수: 0~100 점수를 300~950점 범위로 변환
        credit_score = int(300 + (final_score - 10) * (950 - 300) / (90 - 10))
        credit_score = max(300, min(950, credit_score))
        credit_grade = self.get_credit_grade(credit_score)
        
        return {
            'store_id': row_data.get('store_id', 0),
            'business_registration_no': row_data.get('business_registration_no', ''),
            '신용점수': credit_score,
            '신용등급': credit_grade,
            **{f: int(s) for f, s in factor_scores.items()}
        }
    
    def get_credit_grade(self, score):
        """점수 → 등급 (기존과 동일)"""
        for threshold, grade in self.grade_mapping:
            if score >= threshold:
                return grade
        return "C"
    
    def process_csv(self, input_csv_path, output_csv_path=None):
        """CSV 파일 처리"""
        df = pd.read_csv("/Users/ahnsaeyeon/guide-on/ai/user124.csv", encoding='utf-8', low_memory=False)

        # 먼저 정규화 통계값 계산
        print("정규화 통계값 계산 중...")
        self.fit_normalization(df)
        
        results = []
        for idx, row in df.iterrows():
            try:
                results.append(self.calculate_single_score(row.to_dict()))
                if (idx + 1) % 100 == 0:
                    print(f"처리 완료: {idx + 1}/{len(df)} 행")
            except Exception as e:
                print(f"[오류] 행 {idx}: {e}")
        
        result_df = pd.DataFrame(results)
        
        # 결과 통계 출력
        print(f"\n=== 신용점수 통계 ===")
        print(f"평균: {result_df['신용점수'].mean():.1f}점")
        print(f"중위수: {result_df['신용점수'].median():.1f}점")
        print(f"최고점: {result_df['신용점수'].max()}점")
        print(f"최저점: {result_df['신용점수'].min()}점")
        print(f"\n등급별 분포:")
        print(result_df['신용등급'].value_counts().sort_index())
        
        if not output_csv_path:
            output_csv_path = input_csv_path.replace('.csv', '_신용점수결과.csv')
        
        result_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        print(f"\n결과 저장 완료: {output_csv_path}")
        return result_df

# 실행 예시
if __name__ == "__main__":
    scorer = CSVCreditScorer()
    # 하드코딩된 경로를 매개변수로 변경
    result = scorer.process_csv("user124.csv", "user124_신용점수결과.csv")

정규화 통계값 계산 중...

=== 신용점수 통계 ===
평균: 625.0점
중위수: 625.0점
최고점: 625점
최저점: 625점

등급별 분포:
신용등급
BBB    1
Name: count, dtype: int64

결과 저장 완료: user124_신용점수결과.csv


In [ ]:
import pymysql

conn = pymysql.connect(
    host='localhost',
    port=3306,
    user='root',
    password='Ab1515!!',
    db='guideon',
    charset='utf8')

cursor = conn.cursor(pymysql.cursors.DictCursor)

cursor.execute("select * from store_unified_summary where store_id = 122 and summary_year_month = '2025-08';")

rows = cursor.fetchall()
for r in rows:
    print(r)


{'store_id': 116, 'owner_id': 1116, 'business_registration_no': '116-20-27692', 'industry_code': '56112', 'open_date': datetime.date(2022, 9, 11), 'summary_year_month': '2025-08', 'total_sales_amount': Decimal('3811568.17'), 'weekday_sales_amount': Decimal('2477519.31'), 'weekend_sales_amount': Decimal('1334048.86'), 'lunch_sales_ratio': Decimal('32.63'), 'dinner_sales_ratio': Decimal('44.73'), 'transaction_count': 150, 'weekday_transaction_count': 97, 'weekend_transaction_count': 53, 'mom_growth_rate': Decimal('0.21'), 'yoy_growth_rate': Decimal('3.00'), 'sales_cv': Decimal('0.0611'), 'avg_transaction_value': Decimal('25295.00'), 'weekday_avg_transaction_value': Decimal('23778.00'), 'weekend_avg_transaction_value': Decimal('28331.00'), 'cash_payment_ratio': Decimal('12.50'), 'card_payment_ratio': Decimal('87.50'), 'revisit_customer_sales_ratio': Decimal('72.49'), 'new_customer_ratio': Decimal('34.45'), 'created_dttm': datetime.datetime(2025, 9, 8, 0, 21, 10), 'updated_dttm': datetime.

In [ ]:
import pandas as pd
import numpy as np
import pymysql

class BenchmarkCreditScorer:
    def __init__(self):
        """벤치마크 기반 신용점수 계산기"""
        
        # 요인별 가중치 (기존과 동일)
        self.factor_weights = {
            'Factor_1': {
                'operating_expenses': 0.074535,
                'cost_of_goods_sold': 0.07436,
                'weekend_sales_amount': 0.074554,
                'total_sales_amount': 0.07453,
                'other_expenses': 0.073679,
                'total_salary': 0.0736,
                'rent_expense': 0.072611,
                'weekday_sales_amount': 0.072501,
                'avg_account_balance': 0.07072,
                'operating_profit': 0.069715,
                'weekday_transaction_count': 0.066501,
                'transaction_count': 0.063799,
                'weekend_transaction_count': 0.057164,
                'water_usage_ton': 0.025919,
                'electricity_bill_amount': 0.027752,
                'electricity_usage_kwh': 0.028061
            },
            'Factor_2': {
                'revisit_customer_sales_ratio': 0.083268,
                'sales_cv': 0.084229,
                'new_customer_ratio': 0.078371,
                'operating_profit_ratio': 0.093389,
                'operating_expense_ratio': 0.093389,
                'energy_eff_appliance_ratio': 0.04497,
                'cogs_ratio': 0.049744,
                'customer_review_avg_rating': 0.056357,
                'rent_payment_compliance_rate': 0.025489,
                'card_payment_ratio': 0.057726,
                'cash_payment_ratio': 0.057712,
                'salary_payment_regularity': 0.022855,
                'customer_review_positive_ratio': 0.034197,
                'min_balance_maintenance_ratio': 0.021572,
                'weighted_avg_cash_period': 0.056587,
                'cash_payment_ratio_detail': 0.058045,
                'card_payment_ratio_detail': 0.057432,
                'tax_payment_integrity': 0.012474,
                'cashflow_cv': 0.012193
            },
            'Factor_3': {
                'weekend_avg_transaction_value': 0.229004,
                'weekday_avg_transaction_value': 0.229004,
                'avg_transaction_value': 0.229004,
                'dinner_sales_ratio': 0.131089,
                'lunch_sales_ratio': 0.087794,
                'food_waste_kg_per_day': 0.094106
            },
            'Factor_4': {
                'electricity_usage_kwh': 0.191075,
                'food_waste_kg_per_day': 0.187428,
                'electricity_bill_amount': 0.185198,
                'water_usage_ton': 0.163306,
                'recycle_waste_kg_per_day': 0.160769,
                'gas_usage_m3': 0.112225
            },
            'Factor_5': {
                'cash_payment_ratio': 0.364838,
                'card_payment_ratio': 0.364834,
                'lunch_sales_ratio': 0.270328
            },
            'Factor_6': {
                'card_payment_ratio_detail': 0.341636,
                'cash_payment_ratio_detail': 0.339214,
                'weighted_avg_cash_period': 0.31915
            },
            'Factor_7': {
                'yellow_umbrella_member': 0.484789,
                'yellow_umbrella_amount': 0.362795,
                'yellow_umbrella_months': 0.152416
            },
            'Factor_8': {
                'operating_profit_ratio': 0.435107,
                'operating_expense_ratio': 0.435107,
                'salary_ratio': 0.129785
            }
        }
        
        # 요인별 중요도
        self.factor_importance = {
            'Factor_1': 0.20, 'Factor_2': 0.25,
            'Factor_3': 0.25, 'Factor_4': 0.10,
            'Factor_5': 0.05, 'Factor_6': 0.10,
            'Factor_7': 0.03, 'Factor_8': 0.02
        }
        
        # 역방향 지표
        self.reverse_indicators = {
            'sales_cv', 'cashflow_cv', 'cogs_ratio', 'rent_ratio', 'operating_expenses',
            'operating_expense_ratio', 'weighted_avg_cash_period', 'electricity_usage_kwh',
            'electricity_bill_amount', 'rent_expense', 'gas_usage_m3', 'water_usage_ton',
            'food_waste_kg_per_day', 'other_expenses', 'salary_ratio', 'other_payment_ratio',
            'excessive_withdrawal_frequency', 'origin_price_violation_count'
        }
        
        # 등급 매핑
        self.grade_mapping = [
            (900, "AAA"), (800, "AA"), (700, "A"),
            (600, "BBB"), (500, "BB"), (400, "B"),
            (300, "CCC"), (200, "CC"), (0, "C")
        ]
        
        # 업계 벤치마크 (가상의 업계 평균값들)
        self.industry_benchmarks = {
           # === 재무 비율 지표 (실제 평균 기준) ===
            'operating_profit_ratio': {'good': 25.0, 'average': 21.2, 'poor': 15.0},  # 실제 평균 21.2%
            'operating_expense_ratio': {'good': 75.0, 'average': 78.8, 'poor': 85.0},  # 역방향, 실제 평균 78.8%
            'cogs_ratio': {'good': 42.0, 'average': 46.4, 'poor': 52.0},  # 역방향, 실제 평균 46.4%
            'salary_ratio': {'good': 14.0, 'average': 15.9, 'poor': 18.0},  # 역방향, 실제 평균 15.9%
            'rent_ratio': {'good': 8.0, 'average': 9.3, 'poor': 12.0},  # 역방향, 실제 평균 9.3%
            'other_expenses': {'good': 6.0, 'average': 7.3, 'poor': 10.0},  # 역방향, 매출 대비 %
            
            # === 현금흐름 안정성 지표 ===
            'cashflow_cv': {'good': 0.23, 'average': 0.58, 'poor': 0.93},  # 역방향, 실제 평균 0.58±0.35
            'weighted_avg_cash_period': {'good': 1.0, 'average': 1.4, 'poor': 2.5},  # 역방향, 실제 평균 1.4일
            'excessive_withdrawal_frequency': {'good': 1.0, 'average': 2.1, 'poor': 4.0},  # 역방향, 실제 평균 2.1±3.2회
            
            # === 계좌 관리 지표 ===
            'avg_account_balance': {'good': 15000000, 'average': 9500000, 'poor': 5000000},  # 실제 평균 950만원±1800만원
            'min_balance_maintenance_ratio': {'good': 80.0, 'average': 65.0, 'poor': 40.0},  # 실제 평균 65%±28%
            
            # === 결제 방식 지표 ===
            'cash_payment_ratio': {'good': 40.0, 'average': 52.0, 'poor': 70.0},  # 역방향, 실제 평균 52%
            'cash_payment_ratio_detail': {'good': 40.0, 'average': 52.0, 'poor': 70.0},  # 역방향
            'card_payment_ratio': {'good': 55.0, 'average': 45.0, 'poor': 25.0},  # 실제 평균 45%
            'card_payment_ratio_detail': {'good': 55.0, 'average': 45.0, 'poor': 25.0},
            'other_payment_ratio': {'good': 2.0, 'average': 3.0, 'poor': 5.0},  # 역방향, 실제 평균 3%
            
            # === 납부 신뢰성 지표 ===
            'rent_payment_compliance_rate': {'good': 90.0, 'average': 78.0, 'poor': 55.0},  # 실제 평균 78%±22%
            'utility_payment_compliance_rate': {'good': 87.0, 'average': 74.0, 'poor': 50.0},  # 실제 평균 74%±26%
            'salary_payment_regularity': {'good': 85.0, 'average': 72.0, 'poor': 50.0},  # 실제 평균 72%±24%
            'tax_payment_integrity': {'good': 90.0, 'average': 81.0, 'poor': 65.0},  # 실제 평균 81%±19%
            
            # === 매출 관련 (매출 규모별) ===
            'total_sales_amount': {'good': 15000000, 'average': 10000000, 'poor': 5000000},  # 중규모 기준
            'weekend_sales_amount': {'good': 6000000, 'average': 4000000, 'poor': 2000000},
            'weekday_sales_amount': {'good': 9000000, 'average': 6000000, 'poor': 3000000},
            
            # === 거래 패턴 지표 ===
            'transaction_count': {'good': 1500, 'average': 1000, 'poor': 500},
            'weekday_transaction_count': {'good': 1000, 'average': 650, 'poor': 300},
            'weekend_transaction_count': {'good': 500, 'average': 350, 'poor': 200},
            'avg_transaction_value': {'good': 12000, 'average': 10000, 'poor': 8000},
            'weekday_avg_transaction_value': {'good': 12000, 'average': 10000, 'poor': 8000},
            'weekend_avg_transaction_value': {'good': 12000, 'average': 10000, 'poor': 8000},
            
            # 리뷰 관련
            'customer_review_avg_rating': {'good': 4.5, 'average': 3.8, 'poor': 3.0},
            'customer_review_positive_ratio': {'good': 0.85, 'average': 0.70, 'poor': 0.50},
            
            # 유틸리티
            'electricity_usage_kwh': {'good': 800, 'average': 1200, 'poor': 1800},  # 역방향
            'gas_usage_m3': {'good': 100, 'average': 180, 'poor': 300},  # 역방향
            'water_usage_ton': {'good': 20, 'average': 40, 'poor': 80},  # 역방향
            
            # 폐기물
            'food_waste_kg_per_day': {'good': 5, 'average': 15, 'poor': 30},  # 역방향
            'recycle_waste_kg_per_day': {'good': 2, 'average': 5, 'poor': 10},  # 역방향
        }
    
    def normalize_with_benchmark(self, variable, value):
        """벤치마크 기반 점수 정규화"""
        if pd.isna(value) or np.isinf(value):
            return 50
        
        value = float(value)
        
        # 벤치마크가 있는 경우
        if variable in self.industry_benchmarks:
            benchmark = self.industry_benchmarks[variable]
            good_val = benchmark['good']
            avg_val = benchmark['average']
            poor_val = benchmark['poor']
            
            if variable in self.reverse_indicators:
                # 역방향: 값이 작을수록 좋음
                if value <= good_val:
                    score = 85
                elif value >= poor_val:
                    score = 25
                elif value <= avg_val:
                    # good ~ average 구간: 85 ~ 55점
                    ratio = (value - good_val) / (avg_val - good_val)
                    score = 85 - ratio * 30
                else:
                    # average ~ poor 구간: 55 ~ 25점
                    ratio = (value - avg_val) / (poor_val - avg_val)
                    score = 55 - ratio * 30
            else:
                # 정방향: 값이 클수록 좋음
                if value >= good_val:
                    score = 85
                elif value <= poor_val:
                    score = 25
                elif value >= avg_val:
                    # average ~ good 구간: 55 ~ 85점
                    ratio = (value - avg_val) / (good_val - avg_val)
                    score = 55 + ratio * 30
                else:
                    # poor ~ average 구간: 25 ~ 55점
                    ratio = (value - poor_val) / (avg_val - poor_val)
                    score = 25 + ratio * 30
        else:
            # 벤치마크가 없는 경우: 기본 변환
            if value > 0:
                score = min(85, 30 + np.log10(value + 1) * 10)
            else:
                score = 30
            
            if variable in self.reverse_indicators:
                score = 100 - score
        
        return max(15, min(90, score))
    
    def calculate_factor_score(self, factor_name, row_data):
        """요인 점수 계산"""
        factor_vars = self.factor_weights.get(factor_name, {})
        if not factor_vars:
            return 50
        
        weighted_sum, total_weight = 0, 0
        
        for variable, weight in factor_vars.items():
            if variable in row_data:
                score = self.normalize_with_benchmark(variable, row_data[variable])
                weighted_sum += score * weight
                total_weight += weight
        
        return weighted_sum / total_weight if total_weight > 0 else 50
    
    def calculate_single_score(self, row_data):
        """단일 행 신용점수 계산"""
        factor_scores = {
            f: self.calculate_factor_score(f, row_data)
            for f in self.factor_weights.keys()
        }
        
        # 최종 점수 계산
        total_importance = sum(self.factor_importance.values())
        final_score = sum(
            (factor_scores[f] * self.factor_importance[f]) / total_importance
            for f in factor_scores
        )
        
        # 신용점수: 15~90 점수를  350~950점 범위로 변환
        credit_score = int(350 + (final_score - 15) * (950 - 350) / (90 - 15))
        credit_score = max(350, min(950, credit_score))
        credit_grade = self.get_credit_grade(credit_score)
        
        return {
            'store_id': row_data.get('store_id', 0),
            'business_registration_no': row_data.get('business_registration_no', ''),
            '신용점수': credit_score,
            '신용등급': credit_grade,
            '최종점수_raw': round(final_score, 2),
            **{f: round(s, 2) for f, s in factor_scores.items()}
        }
    
    def get_credit_grade(self, score):
        """점수 → 등급"""
        for threshold, grade in self.grade_mapping:
            if score >= threshold:
                return grade
        return "C"
    
    def process_db_data(self, rows):
        """DB에서 가져온 데이터 처리"""
        results = []
        
        for idx, row in enumerate(rows):
            try:
                # DB 데이터의 Decimal, datetime 등을 적절한 타입으로 변환
                cleaned_row = {}
                for key, value in row.items():
                    if value is None:
                        cleaned_row[key] = None
                    elif isinstance(value, (int, float, str)):
                        cleaned_row[key] = value
                    else:
                        # Decimal, datetime 등을 문자열로 변환 후 필요시 숫자로 변환
                        try:
                            cleaned_row[key] = float(value)
                        except (TypeError, ValueError):
                            cleaned_row[key] = str(value)
                
                result = self.calculate_single_score(cleaned_row)
                results.append(result)
                if (idx + 1) % 100 == 0:
                    print(f"처리 완료: {idx + 1}/{len(rows)} 행")
            except Exception as e:
                print(f"[오류] 행 {idx}: {e}")
                # 오류 발생 시 디버깅을 위해 문제가 된 데이터 출력
                print(f"문제 데이터: {row}")
        
        if not results:
            print("처리된 결과가 없습니다.")
            return pd.DataFrame()
            
        result_df = pd.DataFrame(results)
        
        # 결과 통계 출력
        print(f"\n=== 신용점수 통계 ===")
        print(f"평균: {result_df['신용점수'].mean():.1f}점")
        print(f"중위수: {result_df['신용점수'].median():.1f}점")
        print(f"최고점: {result_df['신용점수'].max()}점")
        print(f"최저점: {result_df['신용점수'].min()}점")
        print(f"\n등급별 분포:")
        print(result_df['신용등급'].value_counts().sort_index())
        
        return result_df
    
    def process_csv(self, input_csv_path, output_csv_path=None):
        """CSV 파일 처리 (기존 메서드 유지)"""
        df = pd.read_csv(input_csv_path, encoding='utf-8', low_memory=False)
        
        results = []
        for idx, row in df.iterrows():
            try:
                results.append(self.calculate_single_score(row.to_dict()))
                if (idx + 1) % 100 == 0:
                    print(f"처리 완료: {idx + 1}/{len(df)} 행")
            except Exception as e:
                print(f"[오류] 행 {idx}: {e}")
        
        result_df = pd.DataFrame(results)
        
        # 결과 통계 출력
        print(f"\n=== 신용점수 통계 ===")
        print(f"평균: {result_df['신용점수'].mean():.1f}점")
        print(f"중위수: {result_df['신용점수'].median():.1f}점")
        print(f"최고점: {result_df['신용점수'].max()}점")
        print(f"최저점: {result_df['신용점수'].min()}점")
        print(f"\n등급별 분포:")
        print(result_df['신용등급'].value_counts().sort_index())
        
        if not output_csv_path:
            output_csv_path = input_csv_path.replace('.csv', '_벤치마크_신용점수결과.csv')
        
        result_df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')
        print(f"\n결과 저장 완료: {output_csv_path}")
        return result_df


# DB 연동 실행 예시
if __name__ == "__main__":
    # DB 연결 및 데이터 가져오기
    conn = pymysql.connect(
        host='localhost',
        port=3306,
        user='root',
        password='Ab1515!!',
        db='guideon',
        charset='utf8'
    )
    
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    cursor.execute("SELECT * FROM store_unified_summary WHERE store_id = 116 AND summary_year_month = '2025-08';")
    rows = cursor.fetchall()
    
    # 신용점수 계산
    scorer = BenchmarkCreditScorer()
    result_df = scorer.process_db_data(rows)
    
    # 결과 출력
    print("\n=== 계산 결과 ===")
    print(result_df)
    
    # 필요하면 CSV로 저장
    result_df.to_csv('db_신용점수결과.csv', index=False, encoding='utf-8-sig')
    
    # DB 연결 종료
    cursor.close()
    conn.close()

: 

In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional, Dict, Any
import pandas as pd
import numpy as np
import pymysql
from decimal import Decimal
import datetime

app = FastAPI()

class BenchmarkCreditScorer:
    def __init__(self):
        """벤치마크 기반 신용점수 계산기"""
        
        # 요인별 가중치 (기존과 동일)
        self.factor_weights = {
            'Factor_1': {
                'operating_expenses': 0.074535,
                'cost_of_goods_sold': 0.07436,
                'weekend_sales_amount': 0.074554,
                'total_sales_amount': 0.07453,
                'other_expenses': 0.073679,
                'total_salary': 0.0736,
                'rent_expense': 0.072611,
                'weekday_sales_amount': 0.072501,
                'avg_account_balance': 0.07072,
                'operating_profit': 0.069715,
                'weekday_transaction_count': 0.066501,
                'transaction_count': 0.063799,
                'weekend_transaction_count': 0.057164,
                'water_usage_ton': 0.025919,
                'electricity_bill_amount': 0.027752,
                'electricity_usage_kwh': 0.028061
            },
            'Factor_2': {
                'revisit_customer_sales_ratio': 0.083268,
                'sales_cv': 0.084229,
                'new_customer_ratio': 0.078371,
                'operating_profit_ratio': 0.093389,
                'operating_expense_ratio': 0.093389,
                'energy_eff_appliance_ratio': 0.04497,
                'cogs_ratio': 0.049744,
                'customer_review_avg_rating': 0.056357,
                'rent_payment_compliance_rate': 0.025489,
                'card_payment_ratio': 0.057726,
                'cash_payment_ratio': 0.057712,
                'salary_payment_regularity': 0.022855,
                'customer_review_positive_ratio': 0.034197,
                'min_balance_maintenance_ratio': 0.021572,
                'weighted_avg_cash_period': 0.056587,
                'cash_payment_ratio_detail': 0.058045,
                'card_payment_ratio_detail': 0.057432,
                'tax_payment_integrity': 0.012474,
                'cashflow_cv': 0.012193
            },
            'Factor_3': {
                'weekend_avg_transaction_value': 0.229004,
                'weekday_avg_transaction_value': 0.229004,
                'avg_transaction_value': 0.229004,
                'dinner_sales_ratio': 0.131089,
                'lunch_sales_ratio': 0.087794,
                'food_waste_kg_per_day': 0.094106
            },
            'Factor_4': {
                'electricity_usage_kwh': 0.191075,
                'food_waste_kg_per_day': 0.187428,
                'electricity_bill_amount': 0.185198,
                'water_usage_ton': 0.163306,
                'recycle_waste_kg_per_day': 0.160769,
                'gas_usage_m3': 0.112225
            },
            'Factor_5': {
                'cash_payment_ratio': 0.364838,
                'card_payment_ratio': 0.364834,
                'lunch_sales_ratio': 0.270328
            },
            'Factor_6': {
                'card_payment_ratio_detail': 0.341636,
                'cash_payment_ratio_detail': 0.339214,
                'weighted_avg_cash_period': 0.31915
            },
            'Factor_7': {
                'yellow_umbrella_member': 0.484789,
                'yellow_umbrella_amount': 0.362795,
                'yellow_umbrella_months': 0.152416
            },
            'Factor_8': {
                'operating_profit_ratio': 0.435107,
                'operating_expense_ratio': 0.435107,
                'salary_ratio': 0.129785
            }
        }
        
        # 요인별 중요도
        self.factor_importance = {
            'Factor_1': 0.20, 'Factor_2': 0.25,
            'Factor_3': 0.25, 'Factor_4': 0.10,
            'Factor_5': 0.05, 'Factor_6': 0.10,
            'Factor_7': 0.03, 'Factor_8': 0.02
        }
        
        # 역방향 지표
        self.reverse_indicators = {
            'sales_cv', 'cashflow_cv', 'cogs_ratio', 'rent_ratio', 'operating_expenses',
            'operating_expense_ratio', 'weighted_avg_cash_period', 'electricity_usage_kwh',
            'electricity_bill_amount', 'rent_expense', 'gas_usage_m3', 'water_usage_ton',
            'food_waste_kg_per_day', 'other_expenses', 'salary_ratio', 'other_payment_ratio',
            'excessive_withdrawal_frequency', 'origin_price_violation_count'
        }
        
        # 등급 매핑
        self.grade_mapping = [
            (900, "AAA"), (800, "AA"), (700, "A"),
            (600, "BBB"), (500, "BB"), (400, "B"),
            (300, "CCC"), (200, "CC"), (0, "C")
        ]
        
        # 업계 벤치마크
        self.industry_benchmarks = {
            'operating_profit_ratio': {'good': 25.0, 'average': 21.2, 'poor': 15.0},
            'operating_expense_ratio': {'good': 75.0, 'average': 78.8, 'poor': 85.0},
            'cogs_ratio': {'good': 42.0, 'average': 46.4, 'poor': 52.0},
            'salary_ratio': {'good': 14.0, 'average': 15.9, 'poor': 18.0},
            'rent_ratio': {'good': 8.0, 'average': 9.3, 'poor': 12.0},
            'other_expenses': {'good': 6.0, 'average': 7.3, 'poor': 10.0},
            'cashflow_cv': {'good': 0.23, 'average': 0.58, 'poor': 0.93},
            'weighted_avg_cash_period': {'good': 1.0, 'average': 1.4, 'poor': 2.5},
            'excessive_withdrawal_frequency': {'good': 1.0, 'average': 2.1, 'poor': 4.0},
            'avg_account_balance': {'good': 15000000, 'average': 9500000, 'poor': 5000000},
            'min_balance_maintenance_ratio': {'good': 80.0, 'average': 65.0, 'poor': 40.0},
            'cash_payment_ratio': {'good': 40.0, 'average': 52.0, 'poor': 70.0},
            'cash_payment_ratio_detail': {'good': 40.0, 'average': 52.0, 'poor': 70.0},
            'card_payment_ratio': {'good': 55.0, 'average': 45.0, 'poor': 25.0},
            'card_payment_ratio_detail': {'good': 55.0, 'average': 45.0, 'poor': 25.0},
            'other_payment_ratio': {'good': 2.0, 'average': 3.0, 'poor': 5.0},
            'rent_payment_compliance_rate': {'good': 90.0, 'average': 78.0, 'poor': 55.0},
            'utility_payment_compliance_rate': {'good': 87.0, 'average': 74.0, 'poor': 50.0},
            'salary_payment_regularity': {'good': 85.0, 'average': 72.0, 'poor': 50.0},
            'tax_payment_integrity': {'good': 90.0, 'average': 81.0, 'poor': 65.0},
            'total_sales_amount': {'good': 15000000, 'average': 10000000, 'poor': 5000000},
            'weekend_sales_amount': {'good': 6000000, 'average': 4000000, 'poor': 2000000},
            'weekday_sales_amount': {'good': 9000000, 'average': 6000000, 'poor': 3000000},
            'transaction_count': {'good': 1500, 'average': 1000, 'poor': 500},
            'weekday_transaction_count': {'good': 1000, 'average': 650, 'poor': 300},
            'weekend_transaction_count': {'good': 500, 'average': 350, 'poor': 200},
            'avg_transaction_value': {'good': 12000, 'average': 10000, 'poor': 8000},
            'weekday_avg_transaction_value': {'good': 12000, 'average': 10000, 'poor': 8000},
            'weekend_avg_transaction_value': {'good': 12000, 'average': 10000, 'poor': 8000},
            'customer_review_avg_rating': {'good': 4.5, 'average': 3.8, 'poor': 3.0},
            'customer_review_positive_ratio': {'good': 0.85, 'average': 0.70, 'poor': 0.50},
            'electricity_usage_kwh': {'good': 800, 'average': 1200, 'poor': 1800},
            'gas_usage_m3': {'good': 100, 'average': 180, 'poor': 300},
            'water_usage_ton': {'good': 20, 'average': 40, 'poor': 80},
            'food_waste_kg_per_day': {'good': 5, 'average': 15, 'poor': 30},
            'recycle_waste_kg_per_day': {'good': 2, 'average': 5, 'poor': 10},
        }
    
    def normalize_with_benchmark(self, variable, value):
        """벤치마크 기반 점수 정규화"""
        # None이나 NaN 체크
        if value is None or pd.isna(value):
            return 50
        
        # Decimal이나 다른 타입을 float로 변환
        try:
            value = float(value)
        except (TypeError, ValueError):
            return 50
        
        # inf 체크
        if np.isinf(value):
            return 50
        
        # 벤치마크가 있는 경우
        if variable in self.industry_benchmarks:
            benchmark = self.industry_benchmarks[variable]
            good_val = benchmark['good']
            avg_val = benchmark['average']
            poor_val = benchmark['poor']
            
            if variable in self.reverse_indicators:
                # 역방향: 값이 작을수록 좋음
                if value <= good_val:
                    score = 85
                elif value >= poor_val:
                    score = 25
                elif value <= avg_val:
                    # good ~ average 구간: 85 ~ 55점
                    ratio = (value - good_val) / (avg_val - good_val)
                    score = 85 - ratio * 30
                else:
                    # average ~ poor 구간: 55 ~ 25점
                    ratio = (value - avg_val) / (poor_val - avg_val)
                    score = 55 - ratio * 30
            else:
                # 정방향: 값이 클수록 좋음
                if value >= good_val:
                    score = 85
                elif value <= poor_val:
                    score = 25
                elif value >= avg_val:
                    # average ~ good 구간: 55 ~ 85점
                    ratio = (value - avg_val) / (good_val - avg_val)
                    score = 55 + ratio * 30
                else:
                    # poor ~ average 구간: 25 ~ 55점
                    ratio = (value - poor_val) / (avg_val - poor_val)
                    score = 25 + ratio * 30
        else:
            # 벤치마크가 없는 경우: 기본 변환
            if value > 0:
                score = min(85, 30 + np.log10(value + 1) * 10)
            else:
                score = 30
            
            if variable in self.reverse_indicators:
                score = 100 - score
        
        return max(15, min(90, score))
    
    def calculate_factor_score(self, factor_name, row_data):
        """요인 점수 계산"""
        factor_vars = self.factor_weights.get(factor_name, {})
        if not factor_vars:
            return 50
        
        weighted_sum, total_weight = 0, 0
        
        for variable, weight in factor_vars.items():
            if variable in row_data:
                score = self.normalize_with_benchmark(variable, row_data[variable])
                weighted_sum += score * weight
                total_weight += weight
        
        return weighted_sum / total_weight if total_weight > 0 else 50
    
    def calculate_single_score(self, row_data):
        """단일 행 신용점수 계산"""
        factor_scores = {
            f: self.calculate_factor_score(f, row_data)
            for f in self.factor_weights.keys()
        }
        
        # 최종 점수 계산
        total_importance = sum(self.factor_importance.values())
        final_score = sum(
            (factor_scores[f] * self.factor_importance[f]) / total_importance
            for f in factor_scores
        )
        
        # 신용점수: 15~90 점수를  350~950점 범위로 변환
        credit_score = int(350 + (final_score - 15) * (950 - 350) / (90 - 15))
        credit_score = max(350, min(950, credit_score))
        credit_grade = self.get_credit_grade(credit_score)
        
        return {
            'store_id': row_data.get('store_id', 0),
            'business_registration_no': row_data.get('business_registration_no', ''),
            '신용점수': credit_score,
            '신용등급': credit_grade,
            '최종점수_raw': round(final_score, 2),
            **{f: round(s, 2) for f, s in factor_scores.items()}
        }
    
    def get_credit_grade(self, score):
        """점수 → 등급"""
        for threshold, grade in self.grade_mapping:
            if score >= threshold:
                return grade
        return "C"

    def clean_db_row(self, row):
        """DB 데이터 정리"""
        cleaned_row = {}
        for key, value in row.items():
            if value is None:
                cleaned_row[key] = None
            elif isinstance(value, (int, float, str)):
                cleaned_row[key] = value
            else:
                # Decimal, datetime 등을 적절히 변환
                try:
                    cleaned_row[key] = float(value)
                except (TypeError, ValueError):
                    cleaned_row[key] = str(value)
        return cleaned_row


# 글로벌 스코어러 인스턴스
scorer = BenchmarkCreditScorer()

class CreditScoreRequest(BaseModel):
    store_id: int
    summary_year_month: str = "2025-08"

class CreditScoreResponse(BaseModel):
    store_id: int
    business_registration_no: str
    신용점수: int
    신용등급: str
    최종점수_raw: float
    Factor_1: float
    Factor_2: float
    Factor_3: float
    Factor_4: float
    Factor_5: float
    Factor_6: float
    Factor_7: float
    Factor_8: float

def get_db_connection():
    """DB 연결"""
    return pymysql.connect(
        host='localhost',
        port=3306,
        user='root',
        password='Ab1515!!',
        db='guideon',
        charset='utf8'
    )

@app.get("/")
async def root():
    return {"message": "신용점수 계산 API"}

@app.post("/hybrid-credit-score-result", response_model=CreditScoreResponse)
async def calculate_credit_score(request: CreditScoreRequest):
    try:
        # DB 연결
        conn = get_db_connection()
        cursor = conn.cursor(pymysql.cursors.DictCursor)
        
        # 데이터 조회
        query = """
        SELECT * FROM store_unified_summary 
        WHERE store_id = %s AND summary_year_month = %s
        """
        cursor.execute(query, (request.store_id, request.summary_year_month))
        row = cursor.fetchone()
        
        if not row:
            raise HTTPException(
                status_code=404, 
                detail=f"Store ID {request.store_id}의 {request.summary_year_month} 데이터를 찾을 수 없습니다."
            )
        
        # 데이터 정리 및 신용점수 계산
        cleaned_row = scorer.clean_db_row(row)
        result = scorer.calculate_single_score(cleaned_row)
        
        # DB 연결 종료
        cursor.close()
        conn.close()
        
        return CreditScoreResponse(**result)
        
    except pymysql.Error as e:
        raise HTTPException(status_code=500, detail=f"데이터베이스 오류: {str(e)}")
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"계산 오류: {str(e)}")

@app.get("/hybrid-credit-score-result/{store_id}")
async def get_credit_score(store_id: int, summary_year_month: str = "2025-08"):
    """GET 방식으로도 조회 가능"""
    request = CreditScoreRequest(store_id=store_id, summary_year_month=summary_year_month)
    return await calculate_credit_score(request)

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop